In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  7 22:08:46 2019

@author: Deechean
"""
from cifar10 import cifar10
import os
import tensorflow as tf
from lenet5 import LetNet5
import tf_general as tfg
import numpy as np


FLAGS = tf.flags.FLAGS
try:
    tf.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 50000, 'epoch')
    tf.flags.DEFINE_integer('batch_size', 512, 'batch size')
    tf.flags.DEFINE_integer('test_size', 5000, 'test size')
    tf.flags.DEFINE_float('lr', 0.01, 'learning rate')
    tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters were defined.')
except:
    print('parameters have been defined.')

CONTINUE = 1

parameters were defined.


In [2]:
data = cifar10();
train_image, train_label = data.get_train_batch(FLAGS.batch_size)
#test_image, test_label = data.get_test_batch(FLAGS.test_size)

train set length: 50000
test image shape: (10000, 32, 32, 3)
test label shape: (10000,)
test set length: 10000


In [3]:
ckpt_dir = 'ckpt/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [4]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 28,28,3], name='x_input')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    y_ = tf.placeholder(tf.int64, [None], name='labels')

In [5]:
with tf.name_scope('prediction'):
    le_net5 = LetNet5(x, keep_prob)
    fc2 = le_net5.fc2
    logits = le_net5.logits
    y = le_net5.prediction

In [6]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                  labels=y_, 
                                                                                  name="cross_entropy_per_example"))

In [7]:
with tf.name_scope('train_step'):
    train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
import time

if __name__ == '__main__':
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(max_to_keep = 1)
        if CONTINUE != 0:
            model_file=tf.train.latest_checkpoint('ckpt/')
            saver.restore(sess,model_file)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess, coord)
        for i in range(9500,FLAGS.epoch):
            train_image, train_label = data.get_train_batch(FLAGS.batch_size)
            _,accuracy_rate = sess.run([train_step,accuracy], feed_dict={keep_prob: 0.5, x:train_image, y_:train_label})
            log = []
            string =time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' iter:' + str(i)+', accuracy='+str(round(accuracy_rate*100,2))+'%'
            log.append(string)
            print(string)
            tfg.saveEvalData('lenet5_cifar.log',log)            
            if (i+1) % 250 == 0:  #保存预测模型
                saver.save(sess,'ckpt/cifar10_'+str(i+1)+'.ckpt',global_step=i)  
                for j in range(2):                    
                    test_image, test_label = data.get_test_batch(FLAGS.test_size)
                    accuracy_rate += sess.run(accuracy,feed_dict={keep_prob: 1, x:test_image, y_:test_label})
                accuracy_rate = accuracy_rate/2
                log = []
                string = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' iter ' + str(i) + ', Test accuracy:' +str(round(accuracy_rate*100,2))+'%'
                print(string)
                log.append(string)
                tfg.saveEvalData('lenet5_cifar.log',log)
    tf.reset_default_graph()

INFO:tensorflow:Restoring parameters from ckpt/cifar10_9500.ckpt-9499
2019-05-23 23:24:10 iter:9500, accuracy=52.73%
2019-05-23 23:24:14 iter:9501, accuracy=56.05%
2019-05-23 23:24:17 iter:9502, accuracy=53.12%
2019-05-23 23:24:20 iter:9503, accuracy=47.46%
2019-05-23 23:24:24 iter:9504, accuracy=54.69%
2019-05-23 23:24:27 iter:9505, accuracy=51.37%
2019-05-23 23:24:31 iter:9506, accuracy=50.39%
2019-05-23 23:24:34 iter:9507, accuracy=50.2%
2019-05-23 23:24:38 iter:9508, accuracy=48.83%
2019-05-23 23:24:41 iter:9509, accuracy=52.54%
2019-05-23 23:24:45 iter:9510, accuracy=52.73%
2019-05-23 23:24:48 iter:9511, accuracy=49.61%
2019-05-23 23:24:51 iter:9512, accuracy=49.22%
2019-05-23 23:24:55 iter:9513, accuracy=51.37%
2019-05-23 23:24:58 iter:9514, accuracy=54.88%
2019-05-23 23:25:02 iter:9515, accuracy=52.34%
2019-05-23 23:25:05 iter:9516, accuracy=51.56%
2019-05-23 23:25:09 iter:9517, accuracy=50.39%
2019-05-23 23:25:12 iter:9518, accuracy=52.34%
2019-05-23 23:25:16 iter:9519, accurac

2019-05-23 23:39:25 iter:9674, accuracy=52.54%
2019-05-23 23:39:30 iter:9675, accuracy=52.93%
2019-05-23 23:39:35 iter:9676, accuracy=54.3%
2019-05-23 23:39:39 iter:9677, accuracy=50.78%
2019-05-23 23:39:44 iter:9678, accuracy=48.05%
2019-05-23 23:39:49 iter:9679, accuracy=50.98%
2019-05-23 23:39:54 iter:9680, accuracy=48.83%
2019-05-23 23:40:00 iter:9681, accuracy=57.81%
2019-05-23 23:40:05 iter:9682, accuracy=50.78%
2019-05-23 23:40:11 iter:9683, accuracy=50.39%
2019-05-23 23:40:17 iter:9684, accuracy=49.8%
2019-05-23 23:40:23 iter:9685, accuracy=51.17%
2019-05-23 23:40:29 iter:9686, accuracy=51.17%
2019-05-23 23:40:37 iter:9687, accuracy=53.32%
2019-05-23 23:40:44 iter:9688, accuracy=50.78%
2019-05-23 23:40:53 iter:9689, accuracy=53.12%
2019-05-23 23:41:03 iter:9690, accuracy=47.27%
2019-05-23 23:41:15 iter:9691, accuracy=50.0%
2019-05-23 23:41:33 iter:9692, accuracy=50.59%
2019-05-23 23:42:15 iter:9693, accuracy=51.95%
2019-05-23 23:43:58 iter:9694, accuracy=54.3%
2019-05-23 23:44:

2019-05-23 23:57:17 iter:9848, accuracy=51.37%
2019-05-23 23:57:20 iter:9849, accuracy=49.8%
2019-05-23 23:57:24 iter:9850, accuracy=52.15%
2019-05-23 23:57:28 iter:9851, accuracy=55.47%
2019-05-23 23:57:32 iter:9852, accuracy=49.61%
2019-05-23 23:57:36 iter:9853, accuracy=50.59%
2019-05-23 23:57:40 iter:9854, accuracy=53.32%
2019-05-23 23:57:44 iter:9855, accuracy=54.49%
2019-05-23 23:57:48 iter:9856, accuracy=48.44%
2019-05-23 23:57:52 iter:9857, accuracy=56.64%
2019-05-23 23:57:57 iter:9858, accuracy=51.76%
2019-05-23 23:58:01 iter:9859, accuracy=57.23%
2019-05-23 23:58:05 iter:9860, accuracy=54.3%
2019-05-23 23:58:09 iter:9861, accuracy=54.1%
2019-05-23 23:58:13 iter:9862, accuracy=50.98%
2019-05-23 23:58:18 iter:9863, accuracy=50.59%
2019-05-23 23:58:22 iter:9864, accuracy=53.52%
2019-05-23 23:58:26 iter:9865, accuracy=51.76%
2019-05-23 23:58:31 iter:9866, accuracy=47.85%
2019-05-23 23:58:35 iter:9867, accuracy=53.12%
2019-05-23 23:58:40 iter:9868, accuracy=53.12%
2019-05-23 23:58

2019-05-24 00:17:16 iter:10022, accuracy=52.93%
2019-05-24 00:17:20 iter:10023, accuracy=53.71%
2019-05-24 00:17:23 iter:10024, accuracy=56.84%
2019-05-24 00:17:27 iter:10025, accuracy=53.91%
2019-05-24 00:17:31 iter:10026, accuracy=49.22%
2019-05-24 00:17:34 iter:10027, accuracy=51.76%
2019-05-24 00:17:38 iter:10028, accuracy=51.95%
2019-05-24 00:17:42 iter:10029, accuracy=56.05%
2019-05-24 00:17:45 iter:10030, accuracy=49.22%
2019-05-24 00:17:49 iter:10031, accuracy=50.0%
2019-05-24 00:17:53 iter:10032, accuracy=52.34%
2019-05-24 00:17:57 iter:10033, accuracy=50.78%
2019-05-24 00:18:00 iter:10034, accuracy=52.73%
2019-05-24 00:18:04 iter:10035, accuracy=52.93%
2019-05-24 00:18:08 iter:10036, accuracy=51.17%
2019-05-24 00:18:12 iter:10037, accuracy=53.71%
2019-05-24 00:18:16 iter:10038, accuracy=53.12%
2019-05-24 00:18:19 iter:10039, accuracy=52.15%
2019-05-24 00:18:23 iter:10040, accuracy=52.54%
2019-05-24 00:18:27 iter:10041, accuracy=51.95%
2019-05-24 00:18:31 iter:10042, accuracy=

2019-05-24 00:36:09 iter:10193, accuracy=49.22%
2019-05-24 00:36:13 iter:10194, accuracy=54.69%
2019-05-24 00:36:16 iter:10195, accuracy=51.56%
2019-05-24 00:36:19 iter:10196, accuracy=53.32%
2019-05-24 00:36:23 iter:10197, accuracy=52.93%
2019-05-24 00:36:26 iter:10198, accuracy=51.37%
2019-05-24 00:36:30 iter:10199, accuracy=52.54%
2019-05-24 00:36:33 iter:10200, accuracy=52.54%
2019-05-24 00:36:37 iter:10201, accuracy=55.86%
2019-05-24 00:36:40 iter:10202, accuracy=52.54%
2019-05-24 00:36:44 iter:10203, accuracy=51.95%
2019-05-24 00:36:47 iter:10204, accuracy=50.59%
2019-05-24 00:36:51 iter:10205, accuracy=50.78%
2019-05-24 00:36:54 iter:10206, accuracy=52.15%
2019-05-24 00:36:58 iter:10207, accuracy=51.76%
2019-05-24 00:37:02 iter:10208, accuracy=54.88%
2019-05-24 00:37:05 iter:10209, accuracy=53.32%
2019-05-24 00:37:09 iter:10210, accuracy=52.34%
2019-05-24 00:37:12 iter:10211, accuracy=57.23%
2019-05-24 00:37:16 iter:10212, accuracy=51.37%
2019-05-24 00:37:19 iter:10213, accuracy

2019-05-24 00:51:23 iter:10363, accuracy=50.39%
2019-05-24 00:51:28 iter:10364, accuracy=49.8%
2019-05-24 00:51:34 iter:10365, accuracy=53.91%
2019-05-24 00:51:39 iter:10366, accuracy=50.2%
2019-05-24 00:51:45 iter:10367, accuracy=53.71%
2019-05-24 00:51:51 iter:10368, accuracy=53.32%
2019-05-24 00:51:57 iter:10369, accuracy=56.64%
2019-05-24 00:52:04 iter:10370, accuracy=53.91%
2019-05-24 00:52:11 iter:10371, accuracy=55.47%
2019-05-24 00:52:19 iter:10372, accuracy=50.59%
2019-05-24 00:52:29 iter:10373, accuracy=54.49%
2019-05-24 00:52:39 iter:10374, accuracy=50.98%
2019-05-24 00:52:53 iter:10375, accuracy=54.3%
2019-05-24 00:53:17 iter:10376, accuracy=58.2%
2019-05-24 00:56:34 iter:10377, accuracy=53.52%
2019-05-24 00:56:37 iter:10378, accuracy=54.1%
2019-05-24 00:56:40 iter:10379, accuracy=51.56%
2019-05-24 00:56:44 iter:10380, accuracy=50.59%
2019-05-24 00:56:47 iter:10381, accuracy=49.8%
2019-05-24 00:56:51 iter:10382, accuracy=56.84%
2019-05-24 00:56:54 iter:10383, accuracy=49.8%

2019-05-24 01:11:17 iter:10534, accuracy=54.3%
2019-05-24 01:11:21 iter:10535, accuracy=54.69%
2019-05-24 01:11:25 iter:10536, accuracy=51.56%
2019-05-24 01:11:29 iter:10537, accuracy=54.49%
2019-05-24 01:11:33 iter:10538, accuracy=51.95%
2019-05-24 01:11:37 iter:10539, accuracy=49.61%
2019-05-24 01:11:41 iter:10540, accuracy=55.27%
2019-05-24 01:11:45 iter:10541, accuracy=51.76%
2019-05-24 01:11:49 iter:10542, accuracy=58.59%
2019-05-24 01:11:54 iter:10543, accuracy=51.37%
2019-05-24 01:11:58 iter:10544, accuracy=50.98%
2019-05-24 01:12:02 iter:10545, accuracy=49.61%
2019-05-24 01:12:06 iter:10546, accuracy=52.93%
2019-05-24 01:12:11 iter:10547, accuracy=53.12%
2019-05-24 01:12:15 iter:10548, accuracy=53.12%
2019-05-24 01:12:19 iter:10549, accuracy=50.59%
2019-05-24 01:12:24 iter:10550, accuracy=51.56%
2019-05-24 01:12:28 iter:10551, accuracy=56.25%
2019-05-24 01:12:33 iter:10552, accuracy=58.4%
2019-05-24 01:12:38 iter:10553, accuracy=53.12%
2019-05-24 01:12:42 iter:10554, accuracy=5

2019-05-24 01:30:14 iter:10706, accuracy=52.34%
2019-05-24 01:30:17 iter:10707, accuracy=54.69%
2019-05-24 01:30:21 iter:10708, accuracy=53.71%
2019-05-24 01:30:25 iter:10709, accuracy=50.98%
2019-05-24 01:30:28 iter:10710, accuracy=50.98%
2019-05-24 01:30:32 iter:10711, accuracy=54.49%
2019-05-24 01:30:36 iter:10712, accuracy=50.78%
2019-05-24 01:30:39 iter:10713, accuracy=56.05%
2019-05-24 01:30:43 iter:10714, accuracy=53.91%
2019-05-24 01:30:47 iter:10715, accuracy=48.05%
2019-05-24 01:30:51 iter:10716, accuracy=55.66%
2019-05-24 01:30:54 iter:10717, accuracy=53.71%
2019-05-24 01:30:58 iter:10718, accuracy=54.49%
2019-05-24 01:31:02 iter:10719, accuracy=57.42%
2019-05-24 01:31:06 iter:10720, accuracy=52.34%
2019-05-24 01:31:10 iter:10721, accuracy=54.3%
2019-05-24 01:31:14 iter:10722, accuracy=55.86%
2019-05-24 01:31:18 iter:10723, accuracy=52.34%
2019-05-24 01:31:22 iter:10724, accuracy=54.3%
2019-05-24 01:31:25 iter:10725, accuracy=52.34%
2019-05-24 01:31:29 iter:10726, accuracy=5

2019-05-24 01:50:39 iter:10876, accuracy=54.1%
2019-05-24 01:50:43 iter:10877, accuracy=55.27%
2019-05-24 01:50:46 iter:10878, accuracy=52.15%
2019-05-24 01:50:50 iter:10879, accuracy=56.25%
2019-05-24 01:50:53 iter:10880, accuracy=50.78%
2019-05-24 01:50:57 iter:10881, accuracy=55.47%
2019-05-24 01:51:00 iter:10882, accuracy=56.05%
2019-05-24 01:51:04 iter:10883, accuracy=51.17%
2019-05-24 01:51:07 iter:10884, accuracy=54.69%
2019-05-24 01:51:11 iter:10885, accuracy=55.66%
2019-05-24 01:51:14 iter:10886, accuracy=47.85%
2019-05-24 01:51:18 iter:10887, accuracy=55.08%
2019-05-24 01:51:21 iter:10888, accuracy=54.69%
2019-05-24 01:51:25 iter:10889, accuracy=49.22%
2019-05-24 01:51:28 iter:10890, accuracy=53.91%
2019-05-24 01:51:32 iter:10891, accuracy=54.1%
2019-05-24 01:51:36 iter:10892, accuracy=53.32%
2019-05-24 01:51:39 iter:10893, accuracy=55.47%
2019-05-24 01:51:43 iter:10894, accuracy=54.69%
2019-05-24 01:51:46 iter:10895, accuracy=48.05%
2019-05-24 01:51:50 iter:10896, accuracy=5